In [ ]:
import urllib
import zipfile
from tempfile import NamedTemporaryFile
import shutil
import datetime as dt

from viresclient import SwarmRequest
import pandas as pd
import cdflib
import matplotlib.pyplot as plt
from matplotlib.dates import DateFormatter
import cartopy.crs as ccrs
import cartopy.feature as cfeature

%load_ext autoreload
%autoreload 2
import utils

In [ ]:
%load_ext watermark
%watermark -i -v -p viresclient,pandas,xarray,matplotlib,cartopy

# 1. Field intensity overview

In [ ]:
# Identify recent 7-day time window to use
today = dt.datetime.combine(dt.date.today(), dt.time())
three_days_ago = today - dt.timedelta(days=3)
ten_days_ago = today - dt.timedelta(days=10)
start, end = ten_days_ago, three_days_ago
start, end

In [ ]:
# Fetch data
request = SwarmRequest()
request.set_collection("SW_OPER_MAGA_LR_1B")
request.set_products(
    measurements=["F"],
    sampling_step="PT1M"
)
data = request.get_between(start, end)
ds = data.as_xarray()
ds

In [ ]:
# Build figure to plot onto
fig = plt.figure(figsize=(10, 5))
ax_Moll = plt.subplot2grid(
    (1, 1), (0, 0), colspan=1,
    projection=ccrs.Mollweide()
)
ax_Moll.add_feature(cfeature.COASTLINE, edgecolor='black', alpha=0.5)

# Draw on the measurements
ds.plot.scatter(
    "Longitude", "Latitude", "F",
    ax=ax_Moll, transform=ccrs.PlateCarree(),
    s=10, alpha=0.4, marker=".",
    cmap="jet", norm=plt.Normalize(20e3, 50e3)
)

# Evaluate model and draw contours
coords = utils.grid(height=430)
IGRF = utils.eval_model(time=start,coords=coords)
utils.plot_contours(
    ax_Moll,
    coords[..., 1], coords[..., 0], IGRF["F"],
    draw_labels=False, linewidths=2, levels=20, alpha=0.5,
    cmap="jet", norm=plt.Normalize(20e3, 50e3)
)

# Prepare descriptive text
ID2NAME = {"A": "ALPHA", "B": "BRAVO", "C": "CHARLIE"}
spacecraft = ID2NAME.get(ds['Spacecraft'].values[0])
spacecraft = f"SWARM {spacecraft}"
altitude = int(round((ds["Radius"].values[0] - 6371200)/1e3, ndigits=-1))
quantity = "Magnetic field intensity (F) / nanoTesla"
sample_rate = "1 Minute"
description = f"""
Spacecraft:   {spacecraft} (~{altitude}km altitude)
Time period:  {start.strftime('%Y-%m-%d')} to {(end-dt.timedelta(days=1)).strftime('%Y-%m-%d')}
Quantity:     {quantity}
Downsampling: {sample_rate}
[ Dots: Measurements, Contours: Model ]
""".strip()
# Place text
fig.text(0, 1.05, description, va="top", fontfamily="monospace", fontsize=12);

# 2. Swarm Dst-like index (MMA Fast-track)

In [ ]:
def fetch_zipped_file(url, file_name):
    "Fetch a given file from an online zip file"
    output_file = NamedTemporaryFile()
    zip_file, _ = urllib.request.urlretrieve(url)
    with zipfile.ZipFile(zip_file, 'r') as zip_ref:
        with zip_ref.open(file_name) as f:
            shutil.copyfileobj(f, output_file)
            output_file.seek(0)
    return output_file

In [ ]:
mma_file_url = "ftp://swarm-diss.eo.esa.int/Level2longterm/MMA/SW_OPER_MMA_SHA_2F_20200101T000000_20200427T223000_0108.ZIP"
mma_file_name = "SW_OPER_MMA_SHA_2F_20200101T000000_20200427T223000_0108.cdf"
mma_file = fetch_zipped_file(mma_file_url, mma_file_name)

In [ ]:
# from viresclient._data_handling import FileReader

# with FileReader(mma_file) as mma_data:
#     print(mma_data.as_pandas_dataframe())
## (currently incompatible)

In [ ]:
mma_data = cdflib.CDF(mma_file.name)
mma_data.cdf_info()

In [ ]:
CDF_EPOCH_1970 = 62167219200000.0

time = mma_data.varget('t_gh')[0]
gh10 = mma_data.varget('qs_geo')[:, :, 0][0]
time, gh10

In [ ]:
time = mma_data.varget('t_gh')[0]
df_mma = pd.DataFrame.from_dict({
    "Timestamp": pd.to_datetime((time - CDF_EPOCH_1970)/1e3, unit="s"),
    "gh_10": 
        mma_data.varget('gh_geo')[:, :, 0][0],
    "qs_10":
        mma_data.varget('qs_geo')[:, :, 0][0]
}
).set_index("Timestamp")
df_mma

In [ ]:
fig, axes = plt.subplots(nrows=1, sharex=True)

ax = axes

def plot_month_week(df, ax):
    df_month = df[(df.index[-1] - dt.timedelta(days=28)):]
    df_week  = df[(df.index[-1] - dt.timedelta(days=7)):]
    df_month.plot(color="grey", ax=ax, legend=False)
    df_week.plot(color="red", ax=ax, legend=False)
    ax.grid()
    ax.set_ylabel("MMA_F q$^1_0$ [nT]")

plot_month_week(df_mma["qs_10"], ax)
# ax.xaxis.set_major_formatter(DateFormatter("%Y-%m-%d"))